In [0]:
import cv2
import numpy as np 
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from scipy.fftpack import fft


In [0]:
!pip install --upgrade imutils
import imutlis

def rotate_image(image):
    # loop over the rotation angles
    for angle in np.arange(0, 360, 15):
	    rotated = imutils.rotate(image, angle)
	    cv2_imshow(rotated)
	    cv2.waitKey(0)
     
    # loop over the rotation angles again, this time ensuring
    # no part of the image is cut off
    for angle in np.arange(0, 360, 5):
        rotated = imutils.rotate_bound(image, angle)
        # rotated_arr = np.mean(rotated, axis=0)
        rotated_mean = np.sum(rotated, axis=0) / np.count_nonzero(rotated, axis=0)
        # oneD_fft(rotated_mean[:256])
        print(angle)
        plt.plot(rotated_mean)
        plt.show()
        # print( np.count_nonzero(rotated, axis=0))
        cv2_imshow(rotated)
        cv2.waitKey(0)


In [0]:
def resize_hist_equalization(img):

    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Resize
    scale_percent = 2
    width = int(img.shape[1] * scale_percent)
    height = int(img.shape[0] * scale_percent)
    dim = (width, height)
    img_resized = cv2.resize(img_gray, dim, interpolation = cv2.INTER_AREA)
    cv2_imshow(img_resized) 

    # Alternative way to find histogram of an image 
    plt.hist(img_gray.ravel(), 256, [0,256]) 
    plt.show() 
    
    # Image equalization
    img_equ = cv2.equalizeHist(img_gray) 
    plt.hist(img_equ.ravel(), 256, [0,256]) 
    plt.show() 
    cv2_imshow(img_equ) 


In [0]:
def morphological_process(img):

    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    kernel = np.ones((3, 3), np.uint8)
    # erosion
    img_erosion = cv2.erode(img, kernel)
    cv2_imshow(img_erosion)

    # dilation
    img_dilation = cv2.dilate(img, kernel)
    cv2_imshow(img_dilation)

    # opening
    img_opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    cv2_imshow(img_opening)

    # closing
    img_closing = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel) 
    cv2_imshow(img_closing)

    # morphological gradient: dilation - erosion = contours
    img_gradient = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel)
    cv2_imshow(img_gradient)

    # tophat = src - opening
    tophat = cv2.morphologyEx(img, cv2.MORPH_TOPHAT, kernel)
    cv2_imshow(tophat)

    # blackhat = closing - src
    blackhat = cv2.morphologyEx(img, cv2.MORPH_BLACKHAT, kernel)
    cv2_imshow(blackhat)


In [0]:
def inverse_color(img):
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    height,width = img.shape
    img_reverse = img.copy()

    for i in range(height):
        for j in range(width):
            # For GRAY_SCALE image
            img_reverse[i,j] = 255 - img[i,j]
            # for R.G.B image: 
            # img_reverse[i,j] = (255-img[i,j][0], 255-img[i,j][1], 255-img[i,j][2])
            
    cv2_imshow(img_reverse)


In [0]:
def hsv_process(img):

    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(img_hsv)

    # lower_blue = np.array([110,50,50])
    # upper_blue = np.array([130,255,255])
    # mask = cv2.inRange(img_hsv, lower_blue, upper_blue) 

    cv2_imshow(h)
    cv2_imshow(s)
    cv2_imshow(v)
    

In [0]:
def rgb_process(img):

    b, g, r = cv2.split(img)
    cv2_imshow(b)
    cv2_imshow(g)
    cv2_imshow(r)


In [0]:
def play_video():

    cap = cv2.VideoCapture('mask.mp4')
    count = 0
    while(cap.isOpened()):
        ret, frame = cap.read()

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        cv2.imwrite("frame%d.jpg" % count, frame)
        cv2_imshow(gray)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        count += 1
        
    cap.release()
    cv2.destroyAllWindows()


In [0]:
def hough_transform(img):
    '''Finds line segments in a binary image using the probabilistic Hough transform.'''

    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_edges = cv2.Canny(img_gray,50,150,apertureSize = 3)
    cv2_imshow(img_edges)

    minLineLength = 100 
    maxLineGap = 5 
    
    # lines – Output vector of lines. Each line is represented by a 4-element vector (x_1, y_1, x_2, y_2)
    # image – 8-bit, single-channel binary image. The image may be modified by the function. better be the canny edges
    # rho – Distance resolution of the accumulator in pixels. e.g., 1.0 
    # theta – Angle resolution of the accumulator in radians. e.g., numpy.pi/180 
    # threshold – Accumulator threshold parameter. Only those lines are returned that get enough votes, e.g.,100
    # minLineLength – Minimum line length. Line segments shorter than that are rejected.
    # maxLineGap – Maximum allowed gap between points on the same line to link them. exceed the gap, take it as one line

    lines = cv2.HoughLinesP(img_edges, 1, np.pi/180, 90, minLineLength, maxLineGap)
    img_black = np.zeros((img_gray.shape[0], img_gray.shape[1]))

    for x in range(0, len(lines)):
        for x1, y1, x2, y2 in lines[x]:
           cv2.line(img_black, (x1, y1), (x2, y2), 255, 1)
    cv2_imshow(img_black)

# https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_houghlines/py_houghlines.html


In [0]:
def find_contours(img):
    
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, img_binary = cv2.threshold (img_gray, 127, 255, cv2.THRESH_BINARY )
    contours, hierarchy = cv2.findContours(img_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    img_black = np.zeros((img_gray.shape[0], img_gray.shape[1], 1), np.uint8)
    cv2.drawContours(img_black, contours, -1, 255, 1) # contourIdx –1, means all the contours are drawn.
    cv2_imshow(img_black) 

    new_contours = [] 
    threshold_area = 100
    for cnt in contours:        
        area = cv2.contourArea(cnt)        
        if area > threshold_area:                   
            new_contours.append(cnt)

    img_black1 = np.zeros((img_gray.shape[0], img_gray.shape[1], 1), np.uint8) #---black in RGB
    cv2.drawContours(img_black1, new_contours, -1, 255, 1) 
    cv2_imshow(img_black1) 


In [0]:
def corner_point(img):
    '''Determines strong corners on an image.'''

    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #img_gray  = np.float32(img_gray)

    corners = cv2.goodFeaturesToTrack(img_gray, 25, 0.01, 10)
    #corners = np.int0(corners)

    for corner in corners:
        x, y = corner.ravel()
        cv2.circle(img, (x, y), 3, (255, 0, 0), -1)

    cv2_imshow(img)


In [0]:
# img = cv2.imread('lenna.png') 
# corner_point(img)